# GitHub Repository Metric Analysis

In this notebook, we will analyze the GitHub data collected for the repositories mentioned in [sigs.yaml](https://github.com/open-services-group/community/blob/main/sigs.yaml) and derive some basic metrics such as number of issues/PR open, number of issues/PR closed, mean time to close PRs/issues etc.

This notebook will serve as a template notebook to analyze different GitHub repositories so that it can be easily executed in automation as part of our metrics processing pipeline. The notebook can be executed in parallel for different repos by passing as an argument the GitHub repository for which we would like to analyze and calculate metrics.

(Related issues: [Issue 1](https://github.com/open-services-group/metrics/issues/19))

In [1]:
import os
import datetime as dt
import numpy as np
from dotenv import find_dotenv, load_dotenv
from matplotlib import pyplot as plt
import warnings
import trino

warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

True

In [2]:
## Create a .env file on your local with the correct configs
repo_slug = os.getenv("REPO_SLUG")
GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")

In [3]:
repo_slug

'os-climate/aicoe-osc-demo'

In [4]:
# Note: The GitHub access token needs to be exported before importing the srcopmetrics package (current bug)
from srcopsmetrics.entities.issue import Issue  # noqa: E402
from srcopsmetrics.entities.pull_request import PullRequest  # noqa: E402

In [5]:
# Gather the data
!python -m srcopsmetrics.cli -clr os-climate/aicoe-osc-demo -e Issue,PullRequest

INFO:srcopsmetrics.github_knowledge:Overall repositories found: 1
INFO:srcopsmetrics.bot_knowledge:######################## Analysing os-climate/aicoe-osc-demo ########################

INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Detected entities:
CodeFrequency # Commit # DependencyUpdate # Fork # Issue # IssueEvent # KebechetUpdateManager # License # PullRequest # PullRequestDiscussion # RawIssue # RawPullRequest # ReadMe # Release # Stargazer # TrafficClones # TrafficPaths # TrafficPaths # TrafficReferrers # TrafficClones # TrafficViews
INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Issue inspection
INFO:srcopsmetrics.entities.tools.storage:Loading knowledge locally
INFO:srcopsmetrics.entities.tools.storage:Data from file %s loaded
INFO:srcopsmetrics.entities.interface:Found previous Issue knowledge for os-climate/aicoe-osc-demo with 77 records
INFO:srcopsmetrics.iterator:-------------Issue An

## Issue Metrics

Now, lets fetch the issues for the repository and derive some metrics.

In [6]:
issue = Issue(repo_slug)
issue_df = issue.load_previous_knowledge(is_local=True)
issue_df.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions
id,,,,,,,,
132,Rerun NLP demo on the new cluster,The new dev cluster with a bigger GPU is up ([...,Shreyanand,2022-02-24 16:12:24,None,NaT,{},{}
131,Use physical-landing bucket for NLP demo,"As an effort to decouple data owned by Trino, ...",Shreyanand,2022-02-24 15:57:35,None,NaT,"{'enhancement': {'color': 'a2eeef', 'labeled_a...",{}
128,Prepare GPU image for training teacher network...,**Is your feature request related to a problem...,pacospace,2022-02-10 16:16:08,erikerlandson,2022-02-17 16:09:37,"{'enhancement': {'color': 'a2eeef', 'labeled_a...",{'pacospace': 8}
127,Value Error related to S3 in demo2 notebook,Value Error related to S3 at Import in when ru...,andraNew,2022-01-07 16:17:07,andraNew,2022-01-13 11:27:54,"{'bug': {'color': 'd73a4a', 'labeled_at': 1641...","{'andraNew': 123, 'erikerlandson': 90, 'chauha..."
125,Create Jupyterbook,Add _toc.yaml and _config.yml for the repo and...,oindrillac,2021-12-17 13:09:41,oindrillac,2021-12-20 13:23:13,{},{}


In [7]:
issues_df = issue_df.reset_index()

In [8]:
issues_df.head()

,id,title,body,created_by,created_at,closed_by,closed_at,labels,interactions
0,132,Rerun NLP demo on the new cluster,The new dev cluster with a bigger GPU is up ([...,Shreyanand,2022-02-24 16:12:24,None,NaT,{},{}
1,131,Use physical-landing bucket for NLP demo,"As an effort to decouple data owned by Trino, ...",Shreyanand,2022-02-24 15:57:35,None,NaT,"{'enhancement': {'color': 'a2eeef', 'labeled_a...",{}
2,128,Prepare GPU image for training teacher network...,**Is your feature request related to a problem...,pacospace,2022-02-10 16:16:08,erikerlandson,2022-02-17 16:09:37,"{'enhancement': {'color': 'a2eeef', 'labeled_a...",{'pacospace': 8}
3,127,Value Error related to S3 in demo2 notebook,Value Error related to S3 at Import in when ru...,andraNew,2022-01-07 16:17:07,andraNew,2022-01-13 11:27:54,"{'bug': {'color': 'd73a4a', 'labeled_at': 1641...","{'andraNew': 123, 'erikerlandson': 90, 'chauha..."
4,125,Create Jupyterbook,Add _toc.yaml and _config.yml for the repo and...,oindrillac,2021-12-17 13:09:41,oindrillac,2021-12-20 13:23:13,{},{}


In [9]:
issue_cols_to_drop = ["labels", "interactions"]
issue_df = issues_df.drop(columns=issue_cols_to_drop)
issue_df.head()

,id,title,body,created_by,created_at,closed_by,closed_at
0,132,Rerun NLP demo on the new cluster,The new dev cluster with a bigger GPU is up ([...,Shreyanand,2022-02-24 16:12:24,None,NaT
1,131,Use physical-landing bucket for NLP demo,"As an effort to decouple data owned by Trino, ...",Shreyanand,2022-02-24 15:57:35,None,NaT
2,128,Prepare GPU image for training teacher network...,**Is your feature request related to a problem...,pacospace,2022-02-10 16:16:08,erikerlandson,2022-02-17 16:09:37
3,127,Value Error related to S3 in demo2 notebook,Value Error related to S3 at Import in when ru...,andraNew,2022-01-07 16:17:07,andraNew,2022-01-13 11:27:54
4,125,Create Jupyterbook,Add _toc.yaml and _config.yml for the repo and...,oindrillac,2021-12-17 13:09:41,oindrillac,2021-12-20 13:23:13


## PR Metrics

Now, lets fetch the PRs for the repository and derive some metrics.

In [10]:
pr = PullRequest(repo_slug)
pr_df = pr.load_previous_knowledge(is_local=True)
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at
id,,,,,,,,,,,,,,,,,,
133,[WIP] Add move data util nb and update config,# Related issues\r\n#131 #132 \r\n\r\nThis PR ...,L,Shreyanand,2022-02-28 21:18:36,NaT,None,NaT,None,1,3,{'review-notebook-app[bot]': 29},{},[],[fcd345815e73d8c8d570a8c32a2a34e3e6bf7447],"[notebooks/demo2/config.py, notebooks/move_dat...",NaT,NaT
130,Fix gpu manifests builds,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XS,pacospace,2022-02-22 09:31:00,NaT,None,NaT,None,1,1,{},{},[],[d87b0a45185451072c2fef554024a273ed8985a3],[manifests/nm-gpu-training-manifests.yaml],NaT,NaT
129,Add manifests for GPU image build,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XL,pacospace,2022-02-14 11:10:27,2022-02-17 16:09:37,erikerlandson,2022-02-17 16:09:37,erikerlandson,1,9,{'pacospace': 22},"{'881638114': {'author': 'erikerlandson', 'wor...",[],[17709f8aac5533bbaa20c311a937c89384644b14],"[manifests/.sops.yaml, manifests/README.md, ma...",2022-02-14 13:36:05,2022-02-14 13:36:05
126,Updated documentation,closes #125 \r\ncloses #110 \r\n\r\nJupyterBoo...,M,oindrillac,2021-12-17 16:40:00,2021-12-20 13:23:14,oindrillac,2021-12-20 13:23:13,oindrillac,1,5,"{'oindrillac': 13, 'chauhankaranraj': 18}","{'835457435': {'author': 'aakankshaduggal', 'w...",[],[523e26606333956764986a296149ca56edf56b40],"[README.md, _config.yml, _toc.yml, notebooks/d...",2021-12-17 17:05:43,2021-12-17 21:18:59
124,Update README,This PR \r\n- updates the README to mention th...,XS,chauhankaranraj,2021-12-14 20:45:08,2021-12-15 12:57:35,MichaelClifford,2021-12-15 12:57:35,MichaelClifford,1,1,{'MichaelClifford': 1},{},[],[ad9668f096e1e5ebc5b123d1c6df4ea5ed98af5a],[notebooks/demo2/README.md],NaT,NaT


In [11]:
pr_df = pr_df.reset_index()

In [12]:
pr_df.head()

,id,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at
0,133,[WIP] Add move data util nb and update config,# Related issues\r\n#131 #132 \r\n\r\nThis PR ...,L,Shreyanand,2022-02-28 21:18:36,NaT,None,NaT,None,1,3,{'review-notebook-app[bot]': 29},{},[],[fcd345815e73d8c8d570a8c32a2a34e3e6bf7447],"[notebooks/demo2/config.py, notebooks/move_dat...",NaT,NaT
1,130,Fix gpu manifests builds,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XS,pacospace,2022-02-22 09:31:00,NaT,None,NaT,None,1,1,{},{},[],[d87b0a45185451072c2fef554024a273ed8985a3],[manifests/nm-gpu-training-manifests.yaml],NaT,NaT
2,129,Add manifests for GPU image build,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XL,pacospace,2022-02-14 11:10:27,2022-02-17 16:09:37,erikerlandson,2022-02-17 16:09:37,erikerlandson,1,9,{'pacospace': 22},"{'881638114': {'author': 'erikerlandson', 'wor...",[],[17709f8aac5533bbaa20c311a937c89384644b14],"[manifests/.sops.yaml, manifests/README.md, ma...",2022-02-14 13:36:05,2022-02-14 13:36:05
3,126,Updated documentation,closes #125 \r\ncloses #110 \r\n\r\nJupyterBoo...,M,oindrillac,2021-12-17 16:40:00,2021-12-20 13:23:14,oindrillac,2021-12-20 13:23:13,oindrillac,1,5,"{'oindrillac': 13, 'chauhankaranraj': 18}","{'835457435': {'author': 'aakankshaduggal', 'w...",[],[523e26606333956764986a296149ca56edf56b40],"[README.md, _config.yml, _toc.yml, notebooks/d...",2021-12-17 17:05:43,2021-12-17 21:18:59
4,124,Update README,This PR \r\n- updates the README to mention th...,XS,chauhankaranraj,2021-12-14 20:45:08,2021-12-15 12:57:35,MichaelClifford,2021-12-15 12:57:35,MichaelClifford,1,1,{'MichaelClifford': 1},{},[],[ad9668f096e1e5ebc5b123d1c6df4ea5ed98af5a],[notebooks/demo2/README.md],NaT,NaT


In [13]:
pr_cols_to_drop = ["interactions", "reviews", "labels", "commits", "changed_files"]
prs_df = pr_df.drop(columns=pr_cols_to_drop)
prs_df.head()

,id,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,first_review_at,first_approve_at
0,133,[WIP] Add move data util nb and update config,# Related issues\r\n#131 #132 \r\n\r\nThis PR ...,L,Shreyanand,2022-02-28 21:18:36,NaT,None,NaT,None,1,3,NaT,NaT
1,130,Fix gpu manifests builds,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XS,pacospace,2022-02-22 09:31:00,NaT,None,NaT,None,1,1,NaT,NaT
2,129,Add manifests for GPU image build,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XL,pacospace,2022-02-14 11:10:27,2022-02-17 16:09:37,erikerlandson,2022-02-17 16:09:37,erikerlandson,1,9,2022-02-14 13:36:05,2022-02-14 13:36:05
3,126,Updated documentation,closes #125 \r\ncloses #110 \r\n\r\nJupyterBoo...,M,oindrillac,2021-12-17 16:40:00,2021-12-20 13:23:14,oindrillac,2021-12-20 13:23:13,oindrillac,1,5,2021-12-17 17:05:43,2021-12-17 21:18:59
4,124,Update README,This PR \r\n- updates the README to mention th...,XS,chauhankaranraj,2021-12-14 20:45:08,2021-12-15 12:57:35,MichaelClifford,2021-12-15 12:57:35,MichaelClifford,1,1,NaT,NaT


In [14]:
_p2smap = {
    "string": "varchar",
    "Float64": "double",
    "Int64": "bigint",
    "datetime64[ns]": "timestamp",
}


def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))


# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0], t=e[1]) for e in pz])

In [17]:
# Create a Trino client
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(
        os.environ["TRINO_USER"], os.environ["TRINO_PSWD"]
    ),
    host=os.environ["TRINO_HOST"],
    port=int(os.environ["TRINO_PORT"]),
    http_scheme="https",
    verify=False,
)
cur = conn.cursor()

In [18]:
cur.execute("show catalogs")
cur.fetchall()[1]

ConnectionError: HTTPSConnectionPool(host='trino.operate-first-cloud', port=443): Max retries exceeded with url: /v1/statement (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd2410defa0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

### Number of issues created over time

In [ ]:
# Let's find the number of issues created daily
issues_created_daily = (
    issues_df["created_at"].groupby(issues_df.created_at.dt.to_period("D")).agg("count")
)

In [ ]:
issues_created_daily.head()

In [ ]:
issues_created_daily.plot.bar()

plt.xlabel("Days")
locs, labels = plt.xticks()
N = 10
plt.xticks(locs[::N], issues_created_daily.index[::N].strftime("%b %Y"))
plt.xticks(rotation=45)
plt.ylabel("# Issues")
plt.title("# Daily Issues Created")
plt.show()

### Number of open issues

In [ ]:
num_open_issues = issues_df["closed_at"].isna().sum()
num_open_issues

### Number of closed issues

In [ ]:
num_close_issues = issues_df["closed_at"].notnull().sum()
num_close_issues

### Mean time to close issues

In [ ]:
# Calculate the time taken to close an issue
issues_df["time_to_close"] = issues_df.closed_at - issues_df.created_at
issues_df.head()

Now let's find out the median time taken to close issues grouped by month.

In [ ]:
issues_closed_monthly = (
    issues_df["time_to_close"]
    .groupby(issues_df.created_at.dt.to_period("M"))
    .agg("median")
)

In [ ]:
issues_closed_monthly.head()

We can visualize the trend in median time to close issues by month. However, in order to best capture all the median values including the outliers we can normalize the data by taking the log of the values before plotting. We should also consider the level of granularity we would like to capture and visualize the median time in i.e. days vs hours vs minutes vs seconds.

Let us first consider the different levels of granularity for the median time to close issues.

In [ ]:
# days
issues_closed_monthly_days = issues_closed_monthly.astype("timedelta64[D]")
# hours
issues_closed_monthly_hours = issues_closed_monthly.astype("timedelta64[h]")
# minutes
issues_closed_monthly_minutes = issues_closed_monthly.astype("timedelta64[m]")
# seconds
issues_closed_monthly_seconds = issues_closed_monthly.astype("timedelta64[s]")

We will now consider the granularity level to be "days" and plot the median time to close issues grouped by months.

In [ ]:
issues_closed_monthly_days.plot()
plt.xlabel("Month")
plt.ylabel("Median time to close (days)")
plt.title("Median Time to Close Issues (Monthly)")
plt.show()

### Number of PRs created over time

In [ ]:
# Let's find the number of PRs created daily
pr_created_daily = (
    pr_df["created_at"].groupby(pr_df.created_at.dt.to_period("D")).agg("count")
)

In [ ]:
pr_created_daily.head()

In [ ]:
pr_created_daily.plot.bar()

plt.xlabel("Days")
locs, labels = plt.xticks()
N = 10
plt.xticks(locs[::N], pr_created_daily.index[::N].strftime("%b %Y"))
plt.xticks(rotation=45)
plt.ylabel("# PRs")
plt.title("# Daily PRs Created")
plt.show()

### Number of open PRs

In [ ]:
num_open_prs = pr_df["closed_at"].isna().sum()
num_open_prs

### Number of closed PRs

In [ ]:
num_close_prs = pr_df["closed_at"].notnull().sum()
num_close_prs

### Ratio of opened to closed PRs over the last 90 days (quarter) 

In [ ]:
num_open_prs_90d = len(
    pr_df[pr_df["created_at"] > (dt.datetime.now() - dt.timedelta(days=90))]
)
num_closed_prs_90d = len(
    pr_df[pr_df["closed_at"] > (dt.datetime.now() - dt.timedelta(days=90))]
)
print("Number of open PRs:", num_open_prs_90d)
print("Number of closed PRs:", num_closed_prs_90d)

ratio = num_open_prs_90d / num_closed_prs_90d
print("Ratio of open to closed PRs over last 90 days:", ratio)

A ratio of 1 indicates that we have managed to close and review all the PRs that were created in the past 90 days. Hence, we should always strive for a ratio of 1.

### Mean time to close PRs

In [ ]:
# Calculate the time taken to close a PR
pr_df["time_to_close"] = pr_df.closed_at - pr_df.created_at
pr_df.head()

Now let's find out the median time taken to close PRs grouped by month.

In [ ]:
prs_closed_monthly = (
    pr_df["time_to_close"].groupby(pr_df.created_at.dt.to_period("M")).agg("median")
)
prs_closed_monthly

We can visualize the trend in median time to close PRs by month. However, in order to best capture all the median values including the outliers we can normalize the data by taking the log of the values before plotting. We should also consider the level of granularity we would like to capture and visualize the median time in i.e. days vs hours vs minutes vs seconds.

Let us first consider the different levels of granularity for the median time to close PRs.

In [ ]:
# days
prs_closed_monthly_days = prs_closed_monthly.astype("timedelta64[D]")
# hours
prs_closed_monthly_hours = prs_closed_monthly.astype("timedelta64[h]")
# minutes
prs_closed_monthly_minutes = prs_closed_monthly.astype("timedelta64[m]")
# seconds
prs_closed_monthly_seconds = prs_closed_monthly.astype("timedelta64[s]")

We can now proceed to normalizing the values

In [ ]:
prs_closed_monthly_days_norm = np.log(prs_closed_monthly_days)
prs_closed_monthly_hours_norm = np.log(prs_closed_monthly_hours)
prs_closed_monthly_minutes_norm = np.log(prs_closed_monthly_minutes)
prs_closed_monthly_seconds_norm = np.log(prs_closed_monthly_seconds)

In [ ]:
prs_closed_monthly_hours_norm

We will now consider the granularity level to be "hours" and plot the median time to close PRs grouped by months.

In [ ]:
prs_closed_monthly_hours_norm.plot()
plt.xlabel("Month")
plt.ylabel("Median time to close (hours)")
plt.title("Median Time to Close PRs (hours)")
plt.show()